In [ ]:
import xarray as xr
import numpy as np
import xesmf as xe

import matplotlib.pyplot as plt
import matplotlib.colors
import cartopy.crs as ccrs
import cmocean

import sys
sys.path.insert(0,'../pyicon-master/') 
import pyicon as pyic

In [2]:
def rad_to_deg(ds):
    """Convert radian units to deg."""
    ### https://gist.github.com/aaronspring/fd54970f71a39e26ab54a8b47170533d
    import numpy as np
    #ds.coords.compute()
    with xr.set_options(keep_attrs=True):
        for c in ds.coords:
            if 'units' in ds[c].attrs:
                if ds[c].attrs['units'] == 'radian':
                    # print(f'convert {c} from rad to deg')
                    ds[c] = ds[c]* 180./np.pi
                    ds[c].attrs['units'] = 'degrees'
            elif 'bnds' in c:
                # print(f'convert {c} from rad to deg')
                ds[c] = ds[c]* 180./np.pi
                ds[c].attrs['units'] = 'degrees'
    return ds

# grid_1deg = xe.util.grid_global(1,1)
grid_1deg = xr.Dataset(
    {"lat": (["lat"], np.arange(-89.5, 89.5+1, 1.)),
     "lon": (["lon"], np.arange(-179.5, 179.5+1, 1.)),
     "lat_b": (["lat_b"], np.arange(-90, 90+1, 1.)),
     "lon_b": (["lon_b"], np.arange(-180, 180+1, 1.))})


def regrid_rec(ds_in):
    ds_in=rad_to_deg(ds_in)
    ds_in=ds_in.rename({'clon':'lon','clat':'lat'})
    ds_out=grid_1deg
    ### https://github.com/pangeo-data/xESMF/issues/115
    ### https://gist.github.com/aaronspring/fd54970f71a39e26ab54a8b47170533d
    re = xe.Regridder(ds_in, ds_out, method='nearest_s2d', locstream_in=True)
    ds_re = re(ds_in)
    
    ds_re.attrs=ds_in.attrs
    for vv in ds_re.data_vars:
        ds_re.data_vars[vv].attrs=ds_in.data_vars[vv].attrs
        
    # ds_in.close()
    # del ds_in
    
    return ds_re

In [92]:
### Calculate atmospheric variables on pressure levels (adopted from atm_circul.py from Hairu Ding)

def height_to_pressure(ds):
    #ds = ds_atm_3d_ctrl
    pfull = ds.pfull.data
    zg = ds.zg.data
    ua = ds.ua.data#[0,:,:]
    va = ds.va.data#[0,:,:]
    ta = ds.ta.data#[0,:,:]
    hus = ds.hus.data#[0,:,:]

    # gridpath = "/work/mh0033/m300602/icon/grids/r2b3_atm_r0030/ckdtree/rectgrids/r2b3_atm_r0030_res1.00_180W-180E_90S-90N.npz"  
    gridpath = "./data/r2b3_atm_r0030_res1.00_180W-180E_90S-90N.npz"

    plevc = np.array([100000,92500,85000,77500,70000,60000,50000,40000,30000,25000,20000,15000,10000,7000,5000,3000,1000])    
    icall, ind_lev, fac = pyic.calc_vertical_interp_weights(pfull[:,:], plevc) #pfull[0,:,:]

    zgvi = zg[ind_lev,icall]*fac+zg[ind_lev+1,icall]*(1.-fac)
    uavi = ua[ind_lev,icall]*fac+ua[ind_lev+1,icall]*(1.-fac)
    vavi = va[ind_lev,icall]*fac+va[ind_lev+1,icall]*(1.-fac)
    tavi = ta[ind_lev,icall]*fac+ta[ind_lev+1,icall]*(1.-fac)
    husvi = hus[ind_lev,icall]*fac+hus[ind_lev+1,icall]*(1.-fac)

    lon, lat, zgvihi = pyic.interp_to_rectgrid(zgvi, gridpath, coordinates='clat clon')
    lon, lat, uavihi = pyic.interp_to_rectgrid(uavi, gridpath, coordinates='clat clon')
    lon, lat, vavihi = pyic.interp_to_rectgrid(vavi, gridpath, coordinates='clat clon')
    lon, lat, tavihi = pyic.interp_to_rectgrid(tavi, gridpath, coordinates='clat clon')
    lon, lat, husvihi = pyic.interp_to_rectgrid(husvi, gridpath, coordinates='clat clon')

    ds_pressure = xr.Dataset(data_vars=dict(
            zg=(['plevc','lat','lon'], zgvihi),
            ua=(['plevc','lat','lon'], uavihi),
            va=(['plevc','lat','lon'], vavihi),
            ta=(['plevc','lat','lon'], tavihi),
            hus=(['plevc','lat','lon'], uavihi)),
        coords=dict(lon=lon,lat=lat,plevc=plevc,),
        attrs=ds.attrs)
    for vv in ds_pressure.data_vars:
        ds_pressure.data_vars[vv].attrs=ds.data_vars[vv].attrs
    
    return ds_pressure

In [ ]:
def overturning_atm(ds):
    ### PyIcon, qp_driver.py, figure: atm_psi
    vaz=ds.va.mean(['lon']) # zonal mean
    plevi = np.concatenate(([107500],0.5*(vaz.plevc[:]+vaz.plevc[:-1]),[0.]))
    dp = np.diff(plevi)
    psi = (-2.*np.pi*6.371e6/9.81)*np.cos(vaz.lat*np.pi/180.)*( (vaz[::-1,:]*dp[::-1,np.newaxis]).cumsum(axis=0) )[::-1,:]*1e-9    
    ### https://fabienmaussion.info/climate_system/week_04/01_Lesson_Wind-Derivatives-Integrals.html
    return psi